In [ ]:
!pip install pandas-ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.6/240.6 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 93.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 97.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 MB 12.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.43.0
    Uninstalling llvmlite-0.43.0:
      Successfully uninstalled llvmlite-0.43.0
  Attempting uninstall: pandas
    Found existing inst

In [ ]:
import pandas as pd
import pandas_ta as ta
import numpy as np
from tqdm import tqdm

In [ ]:
df_domain = pd.read_csv('/content/drive/MyDrive/Data Stock/clear_domain.csv', index_col=0)
df_fundamentals = pd.read_csv('/content/drive/MyDrive/Data Stock/clear_fundamentals.csv', index_col=0)
df_sp500 = pd.read_csv('/content/drive/MyDrive/Data Stock/clear_sp500.csv', index_col=0)

In [ ]:
# Tiền xử lý cột Date
df_sp500['Date'] = pd.to_datetime(df_sp500['Date'])

# Merge df_sp500 với df_domain để thêm thông tin ngành
df_merged = pd.merge(df_sp500, df_domain, on='Symbol', how='left')

# Merge tiếp với df_fundamentals để thêm thông tin cơ bản
df_full = pd.merge(df_merged, df_fundamentals, on='Symbol', how='left')

# Sắp xếp lại để đảm bảo tính tuần tự
df_full = df_full.sort_values(by=['Symbol', 'Date']).reset_index(drop=True)

print(f"Đã kết hợp thành công. Dữ liệu tổng thể có {df_full.shape[0]} dòng và {df_full.shape[1]} cột.")
print("Các cột trong DataFrame tổng thể:")
print(df_full.columns)
print("\nVí dụ 5 dòng đầu tiên của dữ liệu tổng thể:")
print(df_full.head())

Đã kết hợp thành công. Dữ liệu tổng thể có 1614230 dòng và 22 cột.
Các cột trong DataFrame tổng thể:
Index(['Symbol', 'Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume',
       'Dividends', 'Stock Splits', 'GICS Sector', 'forwardPE', 'priceToBook',
       'enterpriseToRevenue', 'profitMargins', 'trailingEps', 'forwardEps',
       'beta', '52WeekChange', 'averageVolume', 'marketCap', 'payoutRatio'],
      dtype='object')

Ví dụ 5 dòng đầu tiên của dữ liệu tổng thể:
  Symbol                      Date       Open       High        Low  \
0      A 2010-09-07 04:00:00+00:00  21.237482  21.251787  20.708155   
1      A 2010-09-08 04:00:00+00:00  20.801144  20.915594  20.593704   
2      A 2010-09-09 04:00:00+00:00  20.922747  21.022890  20.686695   
3      A 2010-09-10 04:00:00+00:00  20.708155  20.872675  20.543634   
4      A 2010-09-13 04:00:00+00:00  20.908442  21.072962  20.708155   

       Close  Adj Close   Volume  Dividends  Stock Splits  ... priceToBook  \
0  20.736767  1

In [ ]:
# A. Tạo các đặc trưng động (Technical Indicators) từ dữ liệu giá
print("Tạo các chỉ báo kỹ thuật...")
df_full.ta.rsi(close='Close', length=14, append=True, groupby='Symbol')
df_full.ta.macd(close='Close', append=True, groupby='Symbol')
df_full.ta.ema(close='Close', length=20, append=True, groupby='Symbol')
df_full.ta.ema(close='Close', length=50, append=True, groupby='Symbol')
df_full.ta.bbands(close='Close', length=20, append=True, groupby='Symbol')
df_full.ta.atr(high='High', low='Low', close='Close', length=14, append=True, groupby='Symbol')
df_full.ta.obv(close='Close', volume='Volume', append=True, groupby='Symbol')

Tạo các chỉ báo kỹ thuật...


,OBV
0,NaN
1,-3.923627e+06
2,-9.864290e+05
3,-3.467460e+06
4,6.679640e+05
...,...
1614225,3.651149e+11
1614226,3.651145e+11
1614227,3.651138e+11
1614228,3.651134e+11


In [ ]:
# B. Tạo các đặc trưng trễ (Lag Features)
print("Tạo các đặc trưng trễ...")
df_full['daily_return'] = df_full.groupby('Symbol')['Close'].pct_change()
for i in range(1, 6):
    df_full[f'daily_return_lag_{i}'] = df_full.groupby('Symbol')['daily_return'].shift(i)

Tạo các đặc trưng trễ...


In [ ]:
# C. Tạo các đặc trưng tương tác (Interaction Features)
print("Tạo các đặc trưng tương tác...")
# Thanh khoản tương đối
df_full['relative_volume'] = df_full['Volume'] / df_full['averageVolume']
# Chuẩn hóa lợi nhuận theo rủi ro
df_full['risk_adjusted_return'] = df_full['daily_return'] / df_full['beta']

Tạo các đặc trưng tương tác...


In [ ]:
# D. Tạo các đặc trưng nhóm (Group-based Features)
print("Tạo các đặc trưng nhóm theo ngành...")
# Tính lợi nhuận trung bình của ngành theo từng ngày
sector_avg_return = df_full.groupby(['Date', 'GICS Sector'])['daily_return'].transform('mean')
# Đặc trưng so sánh lợi nhuận của cổ phiếu với ngành
df_full['return_vs_sector'] = df_full['daily_return'] - sector_avg_return

Tạo các đặc trưng nhóm theo ngành...


In [ ]:
# Làm sạch dữ liệu sau khi tạo đặc trưng
df_featured = df_full.dropna().copy()
print(f"Đã tạo xong đặc trưng. Kích thước dữ liệu cuối cùng: {df_featured.shape}")

Đã tạo xong đặc trưng. Kích thước dữ liệu cuối cùng: (1611619, 44)


In [ ]:
# Đường dẫn đầy đủ đến folder
output_path = "/content/drive/MyDrive/Data Stock/data_with_features.csv"

# Lưu file
df_featured.to_csv(output_path, index=False)

print(f"Lưu file thành công vào '{output_path}'")

Lưu file thành công vào '/content/drive/MyDrive/Data Stock/data_with_features.csv'
